In [4]:
import numpy as np

# import polars as pl
import pandas as pd
import requests
import os
from bpdb import set_trace

In [5]:
file_path = "replays.parquet"
if not os.path.exists(file_path):
    apiUrl = "https://api.bar-rts.com/replays?limit=10000&preset=team&hasBots=true"

    json = requests.get(apiUrl).json()

    data = json["data"]

    # df = pl.from_records(data)
    # df.to_pandas().to_parquet(file_path)
    df_raw = pd.DataFrame.from_records(data)
    df_raw.to_parquet(file_path)
else:
    # df = pl.read_parquet(file_path)
    df_raw = pd.read_parquet(file_path)

AttributeError: module 'pandas' has no attribute 'from_records'

In [ ]:
pd.options.mode.copy_on_write = True


def is_raptors(row):
    for team in row:
        for ai in team["AIs"]:
            if ai["shortName"] == "RaptorsAI":
                return True
    return False


def is_draw(row):
    results = []
    for team in row:
        results.append(team["winningTeam"])
    return all(x == results[0] for x in [team["winningTeam"] for team in row])


def winners(row):
    _winners = []
    for team in row:
        if team["winningTeam"] is True:
            if len(team["Players"]) > 0:
                _winners.extend([player["name"] for player in team["Players"]])
            elif len(team["AIs"]) > 0:
                _winners.extend([ai["shortName"] for ai in team["AIs"]])
    return _winners


def players(row):
    _players = []
    for team in row:
        _players.extend([player["name"] for player in team["Players"]])
    return _players


df_raw["raptors"] = df_raw["AllyTeams"].apply(is_raptors)
df_raw["draw"] = df_raw["AllyTeams"].apply(is_draw)
df_raw["winners"] = df_raw["AllyTeams"].apply(winners)

df_raw["player"] = df_raw["AllyTeams"].apply(players)
df_raw = df_raw.explode("player")
df_raw["won"] = df_raw.apply(lambda row: row.player in row.winners, axis=1)

df_raptors_non_draw = df_raw[
    df_raw["raptors"] & ~df_raw["draw"]
]  # draws might be good to exclude
df_raptors_non_draw["week"] = (
    pd.to_datetime(df_raptors_non_draw["startTime"]).dt.isocalendar().week
)
df = df_raptors_non_draw.groupby(["player", "week"]).agg({"won": "mean", "id": "count"})
df = df.rename(columns={"won": "winrate", "id": "replays"})

df["combined score"] = df.winrate.rank() * 0.4 / len(
    df
) + df.replays.rank() * 0.6 / len(df)
df.sort_values("combined score", inplace=True, ascending=False)

df["combined score"] = (df["combined score"] * 2000).astype(int)

df.winrate = df.winrate * 100
pd.set_option("display.max_rows", 500)
pd.options.display.float_format = "{:.0f}%".format

for week, df_week in df.groupby("week"):
    # df_week = df_week[df_week["replays"] > 4].reset_index()
    df_week = df_week.reset_index()
    df_week.index = np.arange(1, len(df_week) + 1)
    print(df_week.head(10))
    print()

             player  week  winrate  replays  combined score
1    BattleBonerUSA     9      56%       16            1783
2             [zip]     9      60%       10            1755
3              Efan     9      45%       11            1687
4           efraout     9      33%       21            1681
5          Valantor     9      36%       11            1669
6        JoblessBOb     9      30%       20            1657
7              VonS     9      27%       15            1637
8            Erutan     9      60%        5            1635
9   justinplayer201     9      60%        5            1635
10          mr2mark     9      43%        7            1632

            player  week  winrate  replays  combined score
1            Surax    10      54%       13            1771
2       skylazykim    10      56%        9            1743
3        DJK_Okpok    10      56%        9            1743
4   BattleBonerUSA    10      62%        8            1735
5       TA4verBOOM    10      71%        7  